In [15]:
import requests

##### The Odds API Setup
- 500 request limit on Free tier
- Important note: the lines are delayed by several minutes, meaning I may need another scraping tool

In [110]:
API_KEY = "fc35e20b0a8d983f77fc2fa9257a35c7"
SPORTS_URL = f"https://api.the-odds-api.com/v4/sports?apiKey={API_KEY}"
ODDS_URL = f"https://api.the-odds-api.com/v4/sports/basketball_nba/odds/?apiKey={API_KEY}&regions=us&markets=h2h,spreads&oddsFormat=american"


response = requests.get(ODDS_URL)
sportsbook_data = response.json()

In [86]:
nested_first = json_normalize(sportsbook_data, "bookmakers")
json_normalize(nested_first.to_dict(orient="records"))
json_normalize(sportsbook_data)

,id,sport_key,sport_title,commence_time,home_team,away_team,bookmakers
0,b2b80d981ff4426f188472bf545a5325,basketball_nba,NBA,2024-03-06T03:05:57Z,Denver Nuggets,Phoenix Suns,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
1,fdd137330111c44ff0e65e014723480d,basketball_nba,NBA,2024-03-07T00:10:00Z,Washington Wizards,Orlando Magic,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
2,c3dee32711c68f93c75549836115f334,basketball_nba,NBA,2024-03-07T00:40:00Z,Atlanta Hawks,Cleveland Cavaliers,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
3,827bed6b078ff6ffad3e5cb5ff75c01e,basketball_nba,NBA,2024-03-07T00:40:00Z,Houston Rockets,Los Angeles Clippers,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
4,939bc5ec699ebad1d13a780e8a339e7d,basketball_nba,NBA,2024-03-07T00:40:00Z,Philadelphia 76ers,Memphis Grizzlies,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
5,78b5b9c45d27d06e0be2ab258b2ec102,basketball_nba,NBA,2024-03-07T02:10:00Z,Utah Jazz,Chicago Bulls,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
6,ad573989eea644c71f5ba1585e8317d4,basketball_nba,NBA,2024-03-07T03:00:00Z,Golden State Warriors,Milwaukee Bucks,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
7,7f43dbe5080b44ae90836bbcbc659677,basketball_nba,NBA,2024-03-07T03:10:00Z,Portland Trail Blazers,Oklahoma City Thunder,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
8,589eed81cc5543b6147d9b6b1c973c08,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings,"[{'key': 'draftkings', 'title': 'DraftKings', ..."


In [112]:
sportsbook_data

[{'id': 'c7459db77d25feffd478ae5ed85a8e7c',
  'sport_key': 'basketball_nba',
  'sport_title': 'NBA',
  'commence_time': '2024-03-10T02:10:46Z',
  'home_team': 'Denver Nuggets',
  'away_team': 'Utah Jazz',
  'bookmakers': [{'key': 'draftkings',
    'title': 'DraftKings',
    'last_update': '2024-03-10T04:18:40Z',
    'markets': [{'key': 'spreads',
      'last_update': '2024-03-10T04:18:40Z',
      'outcomes': [{'name': 'Denver Nuggets', 'price': -125, 'point': -18.5},
       {'name': 'Utah Jazz', 'price': -105, 'point': 18.5}]}]},
   {'key': 'fanduel',
    'title': 'FanDuel',
    'last_update': '2024-03-10T04:18:18Z',
    'markets': [{'key': 'spreads',
      'last_update': '2024-03-10T04:18:18Z',
      'outcomes': [{'name': 'Denver Nuggets', 'price': 102, 'point': -19.5},
       {'name': 'Utah Jazz', 'price': -132, 'point': 19.5}]}]},
   {'key': 'pointsbetus',
    'title': 'PointsBet (US)',
    'last_update': '2024-03-10T04:17:10Z',
    'markets': [{'key': 'spreads',
      'last_update'

#### Initial Data Ingestion
- Will code most of this in pandas but I need to see if a spark/polars dataframe would work better
long term

In [50]:
import pandas as pd

sportsbook = pd.DataFrame(sportsbook_data)

In [18]:
import polars as pl

pl_sportsbook = pl.DataFrame(sportsbook_data)

In [19]:
# Sample pyspark code

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Create a Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# JSON data
api_response = [
    {"id": 1, "name": "Alice", "age": 25},
    {"id": 2, "name": "Bob", "age": 30},
    {"id": 3, "name": "Charlie", "age": 22},
]

# Define the schema for the DataFrame
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
])

# Create a PySpark DataFrame
df = spark.createDataFrame(api_response, schema=schema)

# Show the DataFrame
df.show()


+---+-------+---+
| id|   name|age|
+---+-------+---+
|  1|  Alice| 25|
|  2|    Bob| 30|
|  3|Charlie| 22|
+---+-------+---+



#### Starting to explore the API

In [70]:
json_normalize(json_normalize(sportsbook["markets"].explode())["outcomes"].explode())

,name,price,point
0,Denver Nuggets,370,NaN
1,Phoenix Suns,-560,NaN
2,Denver Nuggets,-115,9.5
3,Phoenix Suns,-115,-9.5
4,Denver Nuggets,350,NaN
5,Phoenix Suns,-500,NaN
6,Denver Nuggets,-120,9.5
7,Phoenix Suns,-105,-9.5
8,Denver Nuggets,290,NaN
9,Phoenix Suns,-400,NaN


In [46]:
from pandas import json_normalize

sportsbook["bookmakers"][1][1]

{'key': 'pointsbetus',
 'title': 'PointsBet (US)',
 'last_update': '2024-03-06T03:47:00Z',
 'markets': [{'key': 'h2h',
   'last_update': '2024-03-06T03:47:00Z',
   'outcomes': [{'name': 'Denver Nuggets', 'price': -400},
    {'name': 'Phoenix Suns', 'price': 300}]},
  {'key': 'spreads',
   'last_update': '2024-03-06T03:47:00Z',
   'outcomes': [{'name': 'Denver Nuggets', 'price': -112, 'point': -8.0},
    {'name': 'Phoenix Suns', 'price': -112, 'point': 8.0}]}]}

In [36]:
response.headers # This is useful to see how many requests I have left for the month

{'Date': 'Wed, 06 Mar 2024 03:47:23 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '2763', 'Connection': 'keep-alive', 'X-Requests-Used': '2', 'X-Requests-Remaining': '498', 'vary': 'Accept-Encoding', 'content-encoding': 'gzip', 'Apigw-Requestid': 'UMET0hf_IAMEYug='}

In [274]:
import pandas as pd

# Your nested JSON data
data = {
    'id': 'b2b80d981ff4426f188472bf545a5325',
    'sport_key': 'basketball_nba',
    'sport_title': 'NBA',
    'commence_time': '2024-03-06T03:05:57Z',
    'home_team': 'Denver Nuggets',
    'away_team': 'Phoenix Suns',
    'bookmakers': [
        {
            'key': 'draftkings',
            'title': 'DraftKings',
            'last_update': '2024-03-06T04:56:38Z',
            'markets': [
                {
                    'key': 'h2h',
                    'last_update': '2024-03-06T04:56:38Z',
                    'outcomes': [
                        {'name': 'Denver Nuggets', 'price': 130},
                        {'name': 'Phoenix Suns', 'price': -166}
                    ]
                },
                {
                    'key': 'spreads',
                    'last_update': '2024-03-06T04:56:38Z',
                    'outcomes': [
                        {'name': 'Denver Nuggets', 'price': -120, 'point': 2.5},
                        {'name': 'Phoenix Suns', 'price': -110, 'point': -2.5}
                    ]
                }
            ]
        }
    ]
}

# Flatten the nested structure using json_normalize with prefixes
df_bookmakers = pd.json_normalize(sportsbook_data, 'bookmakers', ['id', 'sport_key', 'sport_title', 'commence_time', 'home_team', 'away_team'], record_prefix='bookmaker_')
df_markets = pd.json_normalize(df_bookmakers.to_dict(orient='records'), 'bookmaker_markets', ['id', 'bookmaker_key', 'bookmaker_title', 'bookmaker_last_update'], record_prefix='market_')
df_outcomes = pd.json_normalize(df_markets.to_dict(orient='records'), 'market_outcomes', ['id', 'bookmaker_key', 'market_key', 'market_last_update'], record_prefix='outcome_')

# Combine the DataFrames on common columns
result_df = pd.merge(df_outcomes, df_markets, how='outer', on=['id', 'bookmaker_key', 'market_key', 'market_last_update'])
result_df = pd.merge(result_df, df_bookmakers, how='outer', on=['id', 'bookmaker_key', 'bookmaker_title', 'bookmaker_last_update'])

# Drop unnecessary columns
result_df = result_df.drop(columns=['bookmaker_markets', 'market_outcomes'])

# # Rename columns if needed
# result_df.columns = ['ID', 'Sport_Key', 'Sport_Title', 'Commence_Time', 'Home_Team', 'Away_Team', 'Bookmaker_Key', 'Bookmaker_Title', 'Last_Update', 'Market_Key', 'Market_Last_Update', 'Outcome_Name', 'Outcome_Price', 'Outcome_Point']

# Display the DataFrame
result_df.head()


,outcome_name,outcome_price,outcome_point,id,bookmaker_key,market_key,market_last_update,bookmaker_title,bookmaker_last_update,sport_key,sport_title,commence_time,home_team,away_team
0,Brooklyn Nets,260,NaN,04d7b004a1adc8dce6e33dc3b540a5dd,betmgm,h2h,2024-03-10T04:20:26Z,BetMGM,2024-03-10T04:20:26Z,basketball_nba,NBA,2024-03-10T23:10:00Z,Cleveland Cavaliers,Brooklyn Nets
1,Cleveland Cavaliers,-350,NaN,04d7b004a1adc8dce6e33dc3b540a5dd,betmgm,h2h,2024-03-10T04:20:26Z,BetMGM,2024-03-10T04:20:26Z,basketball_nba,NBA,2024-03-10T23:10:00Z,Cleveland Cavaliers,Brooklyn Nets
2,Brooklyn Nets,-110,7.5,04d7b004a1adc8dce6e33dc3b540a5dd,betmgm,spreads,2024-03-10T04:20:26Z,BetMGM,2024-03-10T04:20:26Z,basketball_nba,NBA,2024-03-10T23:10:00Z,Cleveland Cavaliers,Brooklyn Nets
3,Cleveland Cavaliers,-110,-7.5,04d7b004a1adc8dce6e33dc3b540a5dd,betmgm,spreads,2024-03-10T04:20:26Z,BetMGM,2024-03-10T04:20:26Z,basketball_nba,NBA,2024-03-10T23:10:00Z,Cleveland Cavaliers,Brooklyn Nets
4,Brooklyn Nets,245,NaN,04d7b004a1adc8dce6e33dc3b540a5dd,betrivers,h2h,2024-03-10T04:22:13Z,BetRivers,2024-03-10T04:22:13Z,basketball_nba,NBA,2024-03-10T23:10:00Z,Cleveland Cavaliers,Brooklyn Nets


In [278]:
df_markets

,market_key,market_last_update,market_outcomes,id,bookmaker_key,bookmaker_title,bookmaker_last_update
0,spreads,2024-03-10T04:18:40Z,"[{'name': 'Denver Nuggets', 'price': -125, 'po...",c7459db77d25feffd478ae5ed85a8e7c,draftkings,DraftKings,2024-03-10T04:18:40Z
1,spreads,2024-03-10T04:18:18Z,"[{'name': 'Denver Nuggets', 'price': 102, 'poi...",c7459db77d25feffd478ae5ed85a8e7c,fanduel,FanDuel,2024-03-10T04:18:18Z
2,spreads,2024-03-10T04:17:10Z,"[{'name': 'Denver Nuggets', 'price': -112, 'po...",c7459db77d25feffd478ae5ed85a8e7c,pointsbetus,PointsBet (US),2024-03-10T04:17:10Z
3,spreads,2024-03-10T04:22:12Z,"[{'name': 'Denver Nuggets', 'price': -125, 'po...",c7459db77d25feffd478ae5ed85a8e7c,mybookieag,MyBookie.ag,2024-03-10T04:22:12Z
4,spreads,2024-03-10T04:20:26Z,"[{'name': 'Denver Nuggets', 'price': -175, 'po...",c7459db77d25feffd478ae5ed85a8e7c,betmgm,BetMGM,2024-03-10T04:20:26Z
...,...,...,...,...,...,...,...
185,spreads,2024-03-10T04:20:26Z,"[{'name': 'Los Angeles Lakers', 'price': -110,...",aca24cd0644b7afb7edf2de46f01c241,betmgm,BetMGM,2024-03-10T04:20:26Z
186,h2h,2024-03-10T04:22:13Z,"[{'name': 'Los Angeles Lakers', 'price': -115}...",aca24cd0644b7afb7edf2de46f01c241,betrivers,BetRivers,2024-03-10T04:22:13Z
187,spreads,2024-03-10T04:22:13Z,"[{'name': 'Los Angeles Lakers', 'price': -112,...",aca24cd0644b7afb7edf2de46f01c241,betrivers,BetRivers,2024-03-10T04:22:13Z
188,h2h,2024-03-10T04:22:13Z,"[{'name': 'Los Angeles Lakers', 'price': -115}...",aca24cd0644b7afb7edf2de46f01c241,unibet_us,Unibet,2024-03-10T04:22:13Z


In [163]:
nested_keys = ["test", "test1", "test2"]
nested_key = nested_keys.pop(0)
nested_key

'test'

In [305]:
def find_nested_keys(data: dict):
    for key, value in data.items():
        if isinstance(value, list):
            yield key
            yield from find_nested_keys(value[0])

def find_column_keys(data: dict, nested_keys: list[str]):
    if not nested_keys:
        return
    nested_key = nested_keys.pop(0)
    yield [key for key in data.keys() if not key == nested_key]
    yield from find_column_keys(data[nested_key][0], nested_keys)

# def normalize_json_to_df():


In [207]:
cols = list(find_column_keys(sportsbook_data[0], ["bookmakers", "markets", "outcomes"]))
nested_cols = list(find_nested_keys(sportsbook_data[0]))
print(cols, nested_cols)

[['id', 'sport_key', 'sport_title', 'commence_time', 'home_team', 'away_team'], ['key', 'title', 'last_update'], ['key', 'last_update']] ['bookmakers', 'markets', 'outcomes']


In [279]:
def setup_normalized_tables(data, nested_cols, cols):
    result = []
    for i in range(len(nested_cols)):
        if nested_cols[i] == nested_cols[0]:
            step = pd.json_normalize(data, nested_cols[i], cols[i], record_prefix=f'{nested_cols[i]}_')
            result.append(step)
        else:
            step = pd.json_normalize(result[i-1].to_dict(orient='records'),
                                     f"{nested_cols[i-1]}_{nested_cols[i]}", 
                                     [cols[0][0]] + [f"{nested_cols[i-1]}_{col}" for col in cols[i]], 
                                     record_prefix=f'{nested_cols[i]}_')
            result.append(step)

    for df in result:
        to_drop = [col for col in df.columns if isinstance(df[col][0], list)]
        df.drop(columns=to_drop, inplace=True)
    
    return result




# def setup_normalized_tables(data, nested_cols: list[str], cols: list[list[str]]):
#     if nested_cols[0] in data[0]:
#         yield pd.json_normalize(data, nested_cols[0], cols[0], record_prefix=f'{nested_cols[0]}_')
#     yield from pd.json_normalize(setup_normalized_tables(data, nested_cols, cols).to_dict(orient='records'), f'{nested_cols[0]}_{nested_cols[1]}', [cols[0][0]] + [f"{nested_cols[0]}_{col}" for col in cols[1]], record_prefix=f'{nested_cols[1]}_')


# step1 = pd.json_normalize(sportsbook_data, nested_cols[0], cols[0], record_prefix=f'{nested_cols[0]}_')
# step2 = pd.json_normalize(step1.to_dict(orient='records'), f'{nested_cols[0]}_{nested_cols[1]}', [cols[0][0]] + [f"{nested_cols[0]}_{col}" for col in cols[1]], record_prefix=f'{nested_cols[1]}_')
# step3 = pd.json_normalize(step2.to_dict(orient='records'), f'{nested_cols[1]}_{nested_cols[2]}', [cols[0][0]] + [f"{nested_cols[1]}_{col}" for col in cols[2]], record_prefix=f'{nested_cols[2]}_')

# result_df = pd.merge(df_outcomes, df_markets, how='outer', on=['id', 'bookmaker_key', 'market_key', 'market_last_update'])
# result_df = pd.merge(result_df, df_bookmakers, how='outer', on=['id', 'bookmaker_key', 'bookmaker_title', 'bookmaker_last_update'])

In [327]:
class UndoNest:
    @staticmethod
    def _find_nested_keys(data: dict):
        for key, value in data.items():
            if isinstance(value, list):
                yield key
                yield from UndoNest._find_nested_keys(value[0])

    @staticmethod
    def _find_column_keys(data: dict, nested_keys: list[str]):
        if not nested_keys:
            return
        nested_key = nested_keys.pop(0)
        yield [key for key in data.keys() if not key == nested_key]
        yield from UndoNest._find_column_keys(data[nested_key][0], nested_keys)

    @staticmethod
    def _setup_normalized_tables(data, nested_cols, cols):
        result = []
        for i in range(len(nested_cols)):
            if nested_cols[i] == nested_cols[0]:
                step = pd.json_normalize(data, nested_cols[i], cols[i], record_prefix=f'{nested_cols[i]}_')
                result.append(step)
            else:
                step = pd.json_normalize(result[i-1].to_dict(orient='records'),
                                        f"{nested_cols[i-1]}_{nested_cols[i]}", 
                                        [cols[0][0]] + [f"{nested_cols[i-1]}_{col}" for col in cols[i]], 
                                        record_prefix=f'{nested_cols[i]}_')
                result.append(step)

        for df in result:
            to_drop = [col for col in df.columns if isinstance(df[col][0], list)]
            df.drop(columns=to_drop, inplace=True)
        
        return result

    @staticmethod
    def _backwards_merge(dataframes: list[pd.DataFrame]):
        common_cols = list(set(dataframes[-1].columns) & set(dataframes[-2].columns))
        dataframes[-2] = pd.merge(dataframes[-1], dataframes[-2], how='outer', on=common_cols)
        next_in_line = dataframes[:-1]
        if len(next_in_line) < 2:
            return dataframes[0]
        return UndoNest._backwards_merge(next_in_line)
    
    def __init__(self, json_response: list[dict]):
        if not json_response or not isinstance(json_response, list) or not all(isinstance(d, dict) for d in json_response):
            raise ValueError("Invalid input. Expecting a non-empty list of dictionaries.")
        self.json_response = json_response
        self.nested_keys = list(UndoNest._find_nested_keys(json_response[0]))
        self.column_keys = list(UndoNest._find_column_keys(json_response[0], self.nested_keys.copy()))
        self._normalized_tables = UndoNest._setup_normalized_tables(self.json_response, 
                                                                   self.nested_keys, 
                                                                   self.column_keys)
        self._final_merged_tables = UndoNest._backwards_merge(self._normalized_tables)

    def get_finalized_table(self):
        return self._final_merged_tables

    

In [328]:
unnested_data = UndoNest(sportsbook_data)
unnested_data.get_finalized_table()

,outcomes_name,outcomes_price,outcomes_point,id,markets_key,markets_last_update,bookmakers_key,bookmakers_title,bookmakers_last_update,sport_key,sport_title,commence_time,home_team,away_team
0,Denver Nuggets,100,-17.5,c7459db77d25feffd478ae5ed85a8e7c,spreads,2024-03-10T04:15:17Z,wynnbet,WynnBET,2024-03-10T04:15:17Z,basketball_nba,NBA,2024-03-10T02:10:46Z,Denver Nuggets,Utah Jazz
1,Utah Jazz,-130,17.5,c7459db77d25feffd478ae5ed85a8e7c,spreads,2024-03-10T04:15:17Z,wynnbet,WynnBET,2024-03-10T04:15:17Z,basketball_nba,NBA,2024-03-10T02:10:46Z,Denver Nuggets,Utah Jazz
2,Denver Nuggets,-125,-18.0,c7459db77d25feffd478ae5ed85a8e7c,spreads,2024-03-10T04:16:11Z,betrivers,BetRivers,2024-03-10T04:16:11Z,basketball_nba,NBA,2024-03-10T02:10:46Z,Denver Nuggets,Utah Jazz
3,Utah Jazz,-109,18.0,c7459db77d25feffd478ae5ed85a8e7c,spreads,2024-03-10T04:16:11Z,betrivers,BetRivers,2024-03-10T04:16:11Z,basketball_nba,NBA,2024-03-10T02:10:46Z,Denver Nuggets,Utah Jazz
4,Denver Nuggets,-114,-17.5,c7459db77d25feffd478ae5ed85a8e7c,spreads,2024-03-10T04:16:51Z,unibet_us,Unibet,2024-03-10T04:16:51Z,basketball_nba,NBA,2024-03-10T02:10:46Z,Denver Nuggets,Utah Jazz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1147,Orlando Magic,-106,-1.5,3f74e7082ee8fca0930ad4fecf65277f,spreads,2024-03-10T04:22:15Z,lowvig,LowVig.ag,2024-03-10T04:22:15Z,basketball_nba,NBA,2024-03-10T22:10:00Z,Orlando Magic,Indiana Pacers
1148,Houston Rockets,-106,6.5,6ff4f8186dd54a7f950df1e8b183e2ae,spreads,2024-03-10T04:22:15Z,lowvig,LowVig.ag,2024-03-10T04:22:15Z,basketball_nba,NBA,2024-03-10T22:10:00Z,Sacramento Kings,Houston Rockets
1149,Sacramento Kings,-106,-6.5,6ff4f8186dd54a7f950df1e8b183e2ae,spreads,2024-03-10T04:22:15Z,lowvig,LowVig.ag,2024-03-10T04:22:15Z,basketball_nba,NBA,2024-03-10T22:10:00Z,Sacramento Kings,Houston Rockets
1150,Atlanta Hawks,-106,6.5,9b86cb1a8d38f0936d026ae56302c23a,spreads,2024-03-10T04:22:15Z,lowvig,LowVig.ag,2024-03-10T04:22:15Z,basketball_nba,NBA,2024-03-10T22:10:00Z,Atlanta Hawks,New Orleans Pelicans


In [317]:
# list(find_nested_keys(unnested_data.json_response[0]))
list(unnested_data._find_nested_keys(unnested_data.json_response[0]))

['bookmakers', 'markets', 'outcomes']

In [316]:
list(set(test_list[-1].columns) & set(test_list[-2].columns))

['markets_key',
 'outcomes_price',
 'outcomes_point',
 'markets_last_update',
 'id',
 'outcomes_name']

In [262]:
test_list = [step1, step2, step3]
test_list[-2] = pd.merge(test_list[-1], test_list[-2], how='outer', on=list(set(test_list[-1].columns) & set(test_list[-2].columns)))
next_in_line = test_list[:-1]
if len(next_in_line) < 2:
    print("End.")

next_in_line[-2]

,bookmakers_key,bookmakers_title,bookmakers_last_update,bookmakers_markets,id,sport_key,sport_title,commence_time,home_team,away_team
0,draftkings,DraftKings,2024-03-10T04:18:40Z,"[{'key': 'spreads', 'last_update': '2024-03-10...",c7459db77d25feffd478ae5ed85a8e7c,basketball_nba,NBA,2024-03-10T02:10:46Z,Denver Nuggets,Utah Jazz
1,fanduel,FanDuel,2024-03-10T04:18:18Z,"[{'key': 'spreads', 'last_update': '2024-03-10...",c7459db77d25feffd478ae5ed85a8e7c,basketball_nba,NBA,2024-03-10T02:10:46Z,Denver Nuggets,Utah Jazz
2,pointsbetus,PointsBet (US),2024-03-10T04:17:10Z,"[{'key': 'spreads', 'last_update': '2024-03-10...",c7459db77d25feffd478ae5ed85a8e7c,basketball_nba,NBA,2024-03-10T02:10:46Z,Denver Nuggets,Utah Jazz
3,mybookieag,MyBookie.ag,2024-03-10T04:22:12Z,"[{'key': 'spreads', 'last_update': '2024-03-10...",c7459db77d25feffd478ae5ed85a8e7c,basketball_nba,NBA,2024-03-10T02:10:46Z,Denver Nuggets,Utah Jazz
4,betmgm,BetMGM,2024-03-10T04:20:26Z,"[{'key': 'spreads', 'last_update': '2024-03-10...",c7459db77d25feffd478ae5ed85a8e7c,basketball_nba,NBA,2024-03-10T02:10:46Z,Denver Nuggets,Utah Jazz
...,...,...,...,...,...,...,...,...,...,...
107,pointsbetus,PointsBet (US),2024-03-10T04:22:12Z,"[{'key': 'h2h', 'last_update': '2024-03-10T04:...",aca24cd0644b7afb7edf2de46f01c241,basketball_nba,NBA,2024-03-11T01:40:00Z,Los Angeles Lakers,Minnesota Timberwolves
108,mybookieag,MyBookie.ag,2024-03-10T04:22:12Z,"[{'key': 'h2h', 'last_update': '2024-03-10T04:...",aca24cd0644b7afb7edf2de46f01c241,basketball_nba,NBA,2024-03-11T01:40:00Z,Los Angeles Lakers,Minnesota Timberwolves
109,betmgm,BetMGM,2024-03-10T04:20:26Z,"[{'key': 'h2h', 'last_update': '2024-03-10T04:...",aca24cd0644b7afb7edf2de46f01c241,basketball_nba,NBA,2024-03-11T01:40:00Z,Los Angeles Lakers,Minnesota Timberwolves
110,betrivers,BetRivers,2024-03-10T04:22:13Z,"[{'key': 'h2h', 'last_update': '2024-03-10T04:...",aca24cd0644b7afb7edf2de46f01c241,basketball_nba,NBA,2024-03-11T01:40:00Z,Los Angeles Lakers,Minnesota Timberwolves


In [286]:
backwards_merge(test)

,outcomes_name,outcomes_price,outcomes_point,id,markets_key,markets_last_update,bookmakers_key,bookmakers_title,bookmakers_last_update,sport_key,sport_title,commence_time,home_team,away_team
0,Utah Jazz,-130,17.5,c7459db77d25feffd478ae5ed85a8e7c,spreads,2024-03-10T04:15:17Z,wynnbet,WynnBET,2024-03-10T04:15:17Z,basketball_nba,NBA,2024-03-10T02:10:46Z,Denver Nuggets,Utah Jazz
1,Denver Nuggets,100,-17.5,c7459db77d25feffd478ae5ed85a8e7c,spreads,2024-03-10T04:15:17Z,wynnbet,WynnBET,2024-03-10T04:15:17Z,basketball_nba,NBA,2024-03-10T02:10:46Z,Denver Nuggets,Utah Jazz
2,Denver Nuggets,-125,-18.0,c7459db77d25feffd478ae5ed85a8e7c,spreads,2024-03-10T04:16:11Z,betrivers,BetRivers,2024-03-10T04:16:11Z,basketball_nba,NBA,2024-03-10T02:10:46Z,Denver Nuggets,Utah Jazz
3,Utah Jazz,-109,18.0,c7459db77d25feffd478ae5ed85a8e7c,spreads,2024-03-10T04:16:11Z,betrivers,BetRivers,2024-03-10T04:16:11Z,basketball_nba,NBA,2024-03-10T02:10:46Z,Denver Nuggets,Utah Jazz
4,Utah Jazz,-122,17.5,c7459db77d25feffd478ae5ed85a8e7c,spreads,2024-03-10T04:16:51Z,unibet_us,Unibet,2024-03-10T04:16:51Z,basketball_nba,NBA,2024-03-10T02:10:46Z,Denver Nuggets,Utah Jazz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32235,Indiana Pacers,-106,1.5,3f74e7082ee8fca0930ad4fecf65277f,spreads,2024-03-10T04:22:15Z,lowvig,LowVig.ag,2024-03-10T04:22:15Z,basketball_nba,NBA,2024-03-10T22:10:00Z,Orlando Magic,Indiana Pacers
32236,Sacramento Kings,-106,-6.5,6ff4f8186dd54a7f950df1e8b183e2ae,spreads,2024-03-10T04:22:15Z,lowvig,LowVig.ag,2024-03-10T04:22:15Z,basketball_nba,NBA,2024-03-10T22:10:00Z,Sacramento Kings,Houston Rockets
32237,Houston Rockets,-106,6.5,6ff4f8186dd54a7f950df1e8b183e2ae,spreads,2024-03-10T04:22:15Z,lowvig,LowVig.ag,2024-03-10T04:22:15Z,basketball_nba,NBA,2024-03-10T22:10:00Z,Sacramento Kings,Houston Rockets
32238,New Orleans Pelicans,-106,-6.5,9b86cb1a8d38f0936d026ae56302c23a,spreads,2024-03-10T04:22:15Z,lowvig,LowVig.ag,2024-03-10T04:22:15Z,basketball_nba,NBA,2024-03-10T22:10:00Z,Atlanta Hawks,New Orleans Pelicans


In [281]:
test = setup_normalized_tables(sportsbook_data, nested_cols=nested_cols, cols=cols)
test[0]

,bookmakers_key,bookmakers_title,bookmakers_last_update,id,sport_key,sport_title,commence_time,home_team,away_team
0,draftkings,DraftKings,2024-03-10T04:18:40Z,c7459db77d25feffd478ae5ed85a8e7c,basketball_nba,NBA,2024-03-10T02:10:46Z,Denver Nuggets,Utah Jazz
1,fanduel,FanDuel,2024-03-10T04:18:18Z,c7459db77d25feffd478ae5ed85a8e7c,basketball_nba,NBA,2024-03-10T02:10:46Z,Denver Nuggets,Utah Jazz
2,pointsbetus,PointsBet (US),2024-03-10T04:17:10Z,c7459db77d25feffd478ae5ed85a8e7c,basketball_nba,NBA,2024-03-10T02:10:46Z,Denver Nuggets,Utah Jazz
3,mybookieag,MyBookie.ag,2024-03-10T04:22:12Z,c7459db77d25feffd478ae5ed85a8e7c,basketball_nba,NBA,2024-03-10T02:10:46Z,Denver Nuggets,Utah Jazz
4,betmgm,BetMGM,2024-03-10T04:20:26Z,c7459db77d25feffd478ae5ed85a8e7c,basketball_nba,NBA,2024-03-10T02:10:46Z,Denver Nuggets,Utah Jazz
...,...,...,...,...,...,...,...,...,...
107,pointsbetus,PointsBet (US),2024-03-10T04:22:12Z,aca24cd0644b7afb7edf2de46f01c241,basketball_nba,NBA,2024-03-11T01:40:00Z,Los Angeles Lakers,Minnesota Timberwolves
108,mybookieag,MyBookie.ag,2024-03-10T04:22:12Z,aca24cd0644b7afb7edf2de46f01c241,basketball_nba,NBA,2024-03-11T01:40:00Z,Los Angeles Lakers,Minnesota Timberwolves
109,betmgm,BetMGM,2024-03-10T04:20:26Z,aca24cd0644b7afb7edf2de46f01c241,basketball_nba,NBA,2024-03-11T01:40:00Z,Los Angeles Lakers,Minnesota Timberwolves
110,betrivers,BetRivers,2024-03-10T04:22:13Z,aca24cd0644b7afb7edf2de46f01c241,basketball_nba,NBA,2024-03-11T01:40:00Z,Los Angeles Lakers,Minnesota Timberwolves


In [203]:
test = setup_normalized_tables(sportsbook_data, nested_cols=nested_cols, cols=cols)

KeyError: "Key 'markets' not found. If specifying a record_path, all elements of data should have the path."

,outcomes_name,outcomes_price,outcomes_point,id,markets_key,markets_last_update
0,Denver Nuggets,-125,-18.5,c7459db77d25feffd478ae5ed85a8e7c,spreads,2024-03-10T04:18:40Z
1,Utah Jazz,-105,18.5,c7459db77d25feffd478ae5ed85a8e7c,spreads,2024-03-10T04:18:40Z
2,Denver Nuggets,102,-19.5,c7459db77d25feffd478ae5ed85a8e7c,spreads,2024-03-10T04:18:18Z
3,Utah Jazz,-132,19.5,c7459db77d25feffd478ae5ed85a8e7c,spreads,2024-03-10T04:18:18Z
4,Denver Nuggets,-112,-18.5,c7459db77d25feffd478ae5ed85a8e7c,spreads,2024-03-10T04:17:10Z
...,...,...,...,...,...,...
375,Minnesota Timberwolves,-109,1.0,aca24cd0644b7afb7edf2de46f01c241,spreads,2024-03-10T04:22:13Z
376,Los Angeles Lakers,-115,NaN,aca24cd0644b7afb7edf2de46f01c241,h2h,2024-03-10T04:22:13Z
377,Minnesota Timberwolves,-104,NaN,aca24cd0644b7afb7edf2de46f01c241,h2h,2024-03-10T04:22:13Z
378,Los Angeles Lakers,-112,-1.0,aca24cd0644b7afb7edf2de46f01c241,spreads,2024-03-10T04:22:13Z


In [172]:
df_bookmakers = pd.json_normalize(sportsbook_data, 'bookmakers', ['id', 'sport_key', 'sport_title', 'commence_time', 'home_team', 'away_team'], record_prefix='bookmaker_')
df_markets = pd.json_normalize(df_bookmakers.to_dict(orient='records'), 'bookmaker_markets', ['id', 'bookmaker_key', 'bookmaker_title', 'bookmaker_last_update'], record_prefix='market_')
df_outcomes = pd.json_normalize(df_markets.to_dict(orient='records'), 'market_outcomes', ['id', 'bookmaker_key', 'market_key', 'market_last_update'], record_prefix='outcome_')

In [128]:
find_nested_keys(sportsbook_data[0])

bookmakers
markets
outcomes


In [118]:
for key, value in sportsbook_data[0].items():
    nested_cols = []
    if isinstance(value, list):
        print(key)
        nested_cols.append(key)
        print(nested_cols)
        for key1, value1 in value[0].items():
            if isinstance(value1, list):
                print(key1)
                nested_cols.append(key1)
                print(nested_cols)

bookmakers
['bookmakers']
markets
['bookmakers', 'markets']


In [106]:
# Need to create a dynamic function that can read these nested results without having to manually enter the keys

#### Preview sample dataset, top 40 results for NBA

In [105]:
result_df.head(40)

,outcome_name,outcome_price,outcome_point,id,bookmaker_key,market_key,market_last_update,bookmaker_title,bookmaker_last_update,sport_key,sport_title,commence_time,home_team,away_team
0,Los Angeles Lakers,-145,NaN,589eed81cc5543b6147d9b6b1c973c08,betmgm,h2h,2024-03-06T04:56:19Z,BetMGM,2024-03-06T04:56:19Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
1,Sacramento Kings,120,NaN,589eed81cc5543b6147d9b6b1c973c08,betmgm,h2h,2024-03-06T04:56:19Z,BetMGM,2024-03-06T04:56:19Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
2,Los Angeles Lakers,-110,-2.5,589eed81cc5543b6147d9b6b1c973c08,betmgm,spreads,2024-03-06T04:56:19Z,BetMGM,2024-03-06T04:56:19Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
3,Sacramento Kings,-110,2.5,589eed81cc5543b6147d9b6b1c973c08,betmgm,spreads,2024-03-06T04:56:19Z,BetMGM,2024-03-06T04:56:19Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
4,Los Angeles Lakers,-110,-2.5,589eed81cc5543b6147d9b6b1c973c08,betonlineag,spreads,2024-03-06T04:56:38Z,BetOnline.ag,2024-03-06T04:56:38Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
5,Sacramento Kings,-110,2.5,589eed81cc5543b6147d9b6b1c973c08,betonlineag,spreads,2024-03-06T04:56:38Z,BetOnline.ag,2024-03-06T04:56:38Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
6,Los Angeles Lakers,-137,NaN,589eed81cc5543b6147d9b6b1c973c08,betrivers,h2h,2024-03-06T04:56:38Z,BetRivers,2024-03-06T04:56:38Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
7,Sacramento Kings,116,NaN,589eed81cc5543b6147d9b6b1c973c08,betrivers,h2h,2024-03-06T04:56:38Z,BetRivers,2024-03-06T04:56:38Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
8,Los Angeles Lakers,-109,-2.5,589eed81cc5543b6147d9b6b1c973c08,betrivers,spreads,2024-03-06T04:56:38Z,BetRivers,2024-03-06T04:56:38Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
9,Sacramento Kings,-112,2.5,589eed81cc5543b6147d9b6b1c973c08,betrivers,spreads,2024-03-06T04:56:38Z,BetRivers,2024-03-06T04:56:38Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings


In [107]:
result_df.to_csv("test_nba_results.csv", index=False)